In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [ ]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

dike_model, planning_steps = get_model_for_problem_formulation(3)

In [ ]:
#make sure your pandas is version 0.24 or higher
pd.__version__

In [ ]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=2, policies=2)

In [ ]:
#for unc in dike_model.uncertainties:
#    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [ ]:
#for policy in dike_model.levers:
#    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)


In [ ]:
#for outcome in dike_model.outcomes:
#    print(repr(outcome))

In [ ]:
from ema_workbench import MultiprocessingEvaluator, ema_logging

ema_logging.log_to_stderr(ema_logging.INFO)

n_scenarios = 1000
n_policies = 10

with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, n_policies)

In [ ]:
def problem_formulation_actor(problem_formulation_actor):
   
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    model = Model('dikesnet', function=function)
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE
    
    model.uncertainties = uncertainties
    model.levers = levers

    if problem_formulation_actor == 4: # RWS
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                            variable_name=['{}_Expected Annual Damage'.format(dike)
                                                for dike in function.dikelist],
                            function=np.sum, kind=direction),

            ScalarOutcome('Total Investment Costs',
                            variable_name=['{}_Dike Investment Costs'.format(dike)
                                                for dike in function.dikelist] + ['RfR Total Costs'
                                                                                ] + ['Expected Evacuation Costs'],
                            function=np.sum, kind=direction),

            ScalarOutcome('Expected Number of Deaths',
                            variable_name=['{}_Expected Number of Deaths'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction)] 
    
    elif problem_formulation_actor == 5: # GELDERLAND
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A1-4',
                            variable_name=['A.1_Expected Annual Damage' ,'A.2_Expected Annual Damage',
                                           'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage'],
                          function=sum_over, kind=direction), 

            ScalarOutcome('Investment Costs A1-4',
                            variable_name=['A.1_Dike Investment Costs', 'A.2_Dike Investment Costs',
                                           'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs'], 
                              function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths in A1-4',
                            variable_name=['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths',
                                           'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths'], 
                              function=sum_over, kind=direction)
        ]
    
    elif problem_formulation_actor == 6: # OVERIJSSEL
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A5', variable_name=["A.5_Expected Annual Damage {}".format(i) for i in model.function.planning_steps], function=sum_over, kind=direction),

            ScalarOutcome('Investment Costs A5', variable_name=["A.5_Dike Investment Costs {}".format(i) for i in model.function.planning_steps], function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths in A5', variable_name=["A.5_Expected Number of Deaths {}".format(i) for i in model.function.planning_steps], function=sum_over, kind=direction)]
    
    else:
        raise TypeError('unknown identifier')
    return model

In [ ]:
model = problem_formulation_actor(6)

In [ ]:
model, planning_steps = get_model_for_actor_problem_formulation

In [ ]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
with MultiprocessingEvaluator(model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=50, policies=5)

In [ ]:
cleaned_experiments = experiments.iloc[:,-3:]

In [ ]:
pd.options.display.max_columns = None

In [ ]:
from ema_workbench.analysis import prim

data = outcomes['Investment Costs A5']

y = data < np.percentile(data, 50)

In [ ]:
y

In [ ]:
data

In [ ]:
prim_alg = prim.Prim(cleaned_experiments,y, threshold=0.8)
box1 = prim_alg.find_box()

In [ ]:
box1.show_tradeoff()
plt.show()

In [ ]:
box1.inspect(style='graph')
plt.show()

In [ ]:
box1.inspect(3)
box1.inspect(3, style='graph')
plt.show()

In [ ]:
from ema_workbench.analysis import dimensional_stacking

In [ ]:
x = experiments.iloc[:, 0:26]
data = outcomes['Investment Costs A5']
y = data < np.percentile(data, 30)

In [ ]:
from ema_workbench.analysis import prim
from ema_workbench import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO)

prim_alg = prim.Prim(x, y, threshold=0.8, peel_alpha=0.1)
box1 = prim_alg.find_box()

In [ ]:
box1.show_tradeoff()
plt.show()

In [ ]:
box1.select(2)
box1.show_pairs_scatter()
fig = plt.gcf()
fig.set_size_inches(12,12)
plt.show()

In [ ]:
dimensional_stacking.create_pivot_plot(cleaned_experiments, y)
plt.show()